# Machine Learning Part
* goal of this notebook is to develop code for the machine learning portion
    * dataset loading
    * model
    * evaluation metrics
* code should be added into qdanalysis and refactored aferwards

In [1]:
import tensorflow as tf
from tensorflow import keras

from qdanalysis.models import adapt_resnet50

import pathlib

In [2]:
train_dir = './output/cerug_test_processing/EN'
test_dir = './output/cerug_test_processing/CN'

In [3]:
label_mode = 'categorical'
#shape image will be resized to (figure out ideal values later)
input_shape = (128, 128, 3)
seed = 42
validation_split = 0.2

#due to nature of data, the test and train splits are defined beforehand
train_data = keras.preprocessing.image_dataset_from_directory(train_dir, 
                                                              label_mode=label_mode, 
                                                              image_size=input_shape[:2], 
                                                              seed=seed, 
                                                              validation_split=validation_split, 
                                                              subset='training')
writer_classes = train_data.class_names

Found 1930 files belonging to 2 classes.
Using 1544 files for training.


In [4]:
writer_classes

['Writer0202', 'Writer0909']

In [5]:
validation_data = keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                   label_mode=label_mode,
                                                                   image_size=input_shape[:2],
                                                                   seed=seed,
                                                                   validation_split=validation_split,
                                                                   subset='validation')

Found 1930 files belonging to 2 classes.
Using 386 files for validation.


In [6]:
model = adapt_resnet50(input_shape, len(writer_classes))

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1000)              2049000   
                                                                 
 dense_1 (Dense)             (None, 2)                 2002      
                                                                 
Total params: 25,638,714
Trainable params: 2,051,002
Non-trainable params: 23,587,712
_________________________________________

In [11]:
# optimizer and loss taken from https://stackoverflow.com/questions/71704268/using-tf-keras-utils-image-dataset-from-directory-with-label-list
optimizer = tf.keras.optimizers.Nadam(
    learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-07,
    name='Nadam'
)

lossfn = tf.keras.losses.CategoricalCrossentropy()

metrics = [
    keras.metrics.Accuracy(),
    keras.metrics.Precision(),
    keras.metrics.Recall()
]

model.compile(optimizer=optimizer, loss=lossfn, metrics=metrics)

In [12]:
epochs = 10
model.fit(train_data, epochs=epochs, validation_data=validation_data)

Epoch 1/10


49/49 [==============================] - 28s 497ms/step - loss: 0.7032 - accuracy: 0.0000e+00 - precision_2: 0.5777 - recall_2: 0.5777 - val_loss: 0.6555 - val_accuracy: 0.0000e+00 - val_precision_2: 0.6347 - val_recall_2: 0.6347
Epoch 2/10
49/49 [==============================] - 22s 458ms/step - loss: 0.6573 - accuracy: 0.0000e+00 - precision_2: 0.6282 - recall_2: 0.6282 - val_loss: 0.6469 - val_accuracy: 0.0000e+00 - val_precision_2: 0.6347 - val_recall_2: 0.6347
Epoch 3/10
49/49 [==============================] - 22s 451ms/step - loss: 0.6328 - accuracy: 0.0000e+00 - precision_2: 0.6470 - recall_2: 0.6470 - val_loss: 0.6332 - val_accuracy: 0.0000e+00 - val_precision_2: 0.6503 - val_recall_2: 0.6503
Epoch 4/10
49/49 [==============================] - 22s 454ms/step - loss: 0.6145 - accuracy: 0.0000e+00 - precision_2: 0.6671 - recall_2: 0.6671 - val_loss: 0.6260 - val_accuracy: 0.0000e+00 - val_precision_2: 0.6632 - val_recall_2: 0.6632
Epoch 5/10
49/49 [=============================